# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Extacting csv data from a file for use in Pandas
vaca_df=pd.read_csv("output_data/output.csv")
vaca_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Cidreira,-30.1811,-50.2056,76.62,81,100,5.88,BR,1613334721
1,Hermanus,-34.4187,19.2345,63.00,93,80,1.01,ZA,1613334722
2,Puerto Ayora,-0.7393,-90.3518,81.00,74,90,9.22,EC,1613334669
3,Geraldton,-28.7667,114.6000,69.80,83,14,4.43,AU,1613334653
4,Rikitea,-23.1203,-134.9692,80.13,76,52,16.11,PF,1613334722


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Created variables for the DF locations of Lat, Lng, and Humidity and changed them to (float) data type
locations = vaca_df[["Lat", "Lng"]].astype(float)
Humid = vaca_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=Humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
hotel_df=vaca_df.loc[(vaca_df['Max Temp']>70) & (vaca_df['Max Temp']<80) & (vaca_df['Wind Speed']<10)
                       & (vaca_df['Cloudiness']==0) & (vaca_df['Humidity']>30)].copy()

hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
42,Salalah,17.0151,54.0924,71.60,64,0,4.61,OM,1613334730
144,Chabahar,25.2919,60.6430,70.12,68,0,9.46,IR,1613334751
322,Arteaga,18.4667,-102.4167,72.99,42,0,7.65,MX,1613334785
545,Illapel,-31.6308,-71.1653,78.22,37,0,8.72,CL,1613334827


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df['Hotel Name']=""
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
42,Salalah,17.0151,54.0924,71.60,64,0,4.61,OM,1613334730,
144,Chabahar,25.2919,60.6430,70.12,68,0,9.46,IR,1613334751,
322,Arteaga,18.4667,-102.4167,72.99,42,0,7.65,MX,1613334785,
545,Illapel,-31.6308,-71.1653,78.22,37,0,8.72,CL,1613334827,


In [6]:
# set up a parameters dictionary
params = {
    "location": "",
    "radius": 5000,
    "type": 'lodging',
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for i in range (len(hotel_df)):
    params['location']= f'{hotel_df.iloc[i,1]},{hotel_df.iloc[i,2]}'
# run a request using our params dictionary
    response = requests.get(base_url, params=params)
    hotel_data = response.json()
    hotel_df.iloc[i,-1]= hotel_data['results'][0]['name']

IndexError: list index out of range

In [10]:
#Print the json (pretty printed)
print(json.dumps(hotel_data, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}


In [7]:
hotel_df.to_csv("output_data/vaca_output.csv")
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
42,Salalah,17.0151,54.0924,71.60,64,0,4.61,OM,1613334730,HAMDAN PLAZA HOTEL SALALAH
144,Chabahar,25.2919,60.6430,70.12,68,0,9.46,IR,1613334751,هتل گدروشیا
322,Arteaga,18.4667,-102.4167,72.99,42,0,7.65,MX,1613334785,
545,Illapel,-31.6308,-71.1653,78.22,37,0,8.72,CL,1613334827,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template =""""
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
hotel_info

In [ ]:
# Add marker layer ontop of heat map
marker_locations = hotel_df[['Lat', 'Lng']]

fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_template)
fig.add_layer(markers)
fig

